In [ ]:
import ir_datasets
from datetime import datetime
from src.dataloader import IrDataset
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator
from sentence_transformers import util

Param

In [ ]:
model_name = "microsoft/MiniLM-L12-H384-uncased"
train_batch_size = 32
num_epochs = 1
model_save_path = (
    "output/training_ms-marco_cross-encoder-"
    + model_name.replace("/", "-")
    + "-"
    + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)


Data

In [ ]:
train_dataset = IrDataset(ir_datasets.load("msmarco-passage/train/judged"))
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size)

eval_dataset1 = IrDataset(ir_datasets.load("msmarco-passage/trec-dl-2019/judged"))
eval_dataset2 = IrDataset(ir_datasets.load("msmarco-passage/trec-dl-2020/judged"))


def create_samples(dataset: IrDataset):
    queries = dataset.query_iter
    docs = dataset.docs_iter
    samples = {}
    for i in range(len(dataset)):
        samples[qid] = {"query": queries[qid], "positive": set(), "negative": set()}


# Bon j'arrive pas trop, j'vais copier le code du gars
# mais nous yet on doit le faire l'eval qui se passe pendant le train sur les deux trec

In [ ]:
import os, gzip
data_folder = "./data"
train_eval_filepath = os.path.join(data_folder, 'msmarco-qidpidtriples.rnd-shuf.train-eval.tsv.gz')
if not os.path.exists(train_eval_filepath):
    print("Download "+os.path.basename(train_eval_filepath))
    util.http_get('https://sbert.net/datasets/msmarco-qidpidtriples.rnd-shuf.train-eval.tsv.gz', train_eval_filepath)

dev_samples = {}
with gzip.open(train_eval_filepath, 'rt') as fIn:
    for line in fIn:
        qid, pos_id, neg_id = line.strip().split()

        if qid not in dev_samples and len(dev_samples) < num_dev_queries:
            dev_samples[qid] = {'query': queries[qid], 'positive': set(), 'negative': set()}

        if qid in dev_samples:
            dev_samples[qid]['positive'].add(corpus[pos_id])

            if len(dev_samples[qid]['negative']) < num_max_dev_negatives:
                dev_samples[qid]['negative'].add(corpus[neg_id])
                
=> Bon en

model

In [ ]:
model = CrossEncoder(model_name, num_labels=1, max_length=512)
evaluator = CERerankingEvaluator(dev_samples, name="train-eval")
